## JSON

In the example below, the data from a JSon file is read directly as a DataFrame.

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
filename = '../data/nobelprize.json'
if not os.path.exists(filename):
    import urllib.request
    url = 'http://api.nobelprize.org/v1/prize.json'
    f = urllib.request.urlretrieve (url, filename)

In [ ]:
prizes.printSchema()

Often JSON files have a single root containing multiple records as an array. Inconveniently, `printSchema` shows the name `root` while the actual name in the data is `prizes`. We can `flatten` the data so that instead of one entry that consists of an array, every prize becomes a separate entry.

In [ ]:
df = sqlContext.read.json("../data/nobelprize.json")
prizes = df.select(explode("prizes")).selectExpr("col as prize")
prizes.createOrReplaceTempView("prizes")

Note that in this JSon file year is inferred as a String, therefore, if we want to use numeric expressions on year we need to convert it to int.

In [ ]:
sqlDF = spark.sql("select prize.category, int(prize.year) + 1\
                   as year from prizes")

In [ ]:
sqlDF.show(8)

### use a selectExpr to cast values ###
However, when we use SQL functions, values are automatically cast to their required type.

In [ ]:
sqlDF.selectExpr("year * 2 as y", "year - 2 as x").show(7)

Every prize can have an array of laureates. If we want to process every indivual laureate, we can use explode to duplicate the prize entries for every laureate.

In [ ]:
flatPrizes = prizes.select(prizes['prize.year'],\
                           prizes['prize.category'], \
                           explode(prizes['prize.laureates']).alias('l'))
flatterPrizes = flatPrizes.select(flatPrizes['year'],\
                                  flatPrizes['category'],
                                  flatPrizes['l.id'].alias('pid'),
                                  flatPrizes['l.firstname'],
                                  flatPrizes['l.surname'],
                                  flatPrizes['l.motivation'])
flatterPrizes.createOrReplaceTempView("laureates")
flatterPrizes.show(5)

In [ ]:
spark.sql("select category, year, surname from laureates").show(6)